In [30]:
import pandas as pd
import PIL
import cv2
from PIL import Image
from matplotlib import image

In [2]:
data = pd.read_csv("C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/HAM10000_metadata.csv")

In [3]:
data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


> Dataframe containing labels

In [4]:
data_2 = pd.read_csv("C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/hmnist_28_28_RGB.csv")

In [5]:
label = data_2["label"]

> New dataframe with both image titles and labels:

In [6]:
data = pd.merge(data, label, left_index= True, right_index= True)

In [7]:
data["dx"].value_counts()

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [8]:
data["label"].value_counts()

4    6705
6    1113
2    1099
1     514
0     327
5     142
3     115
Name: label, dtype: int64

In [9]:
#taking 50% of overall data, by category, for the training df
df4_train = data[data["label"] == 4].sample(frac = .5)
df6_train = data[data["label"] == 6].sample(frac = .5)
df2_train = data[data["label"] == 2].sample(frac = .5)
df1_train = data[data["label"] == 1].sample(frac = .5)
df0_train = data[data["label"] == 0].sample(frac = .5)
df5_train = data[data["label"] == 5].sample(frac = .5)
df3_train = data[data["label"] == 3].sample(frac = .5)

In [10]:
#val df is created from the remainder of the original data
val_test_df = data.drop(df4_train.index)
val_test_df.drop(df6_train.index, inplace = True)
val_test_df.drop(df2_train.index, inplace = True)
val_test_df.drop(df1_train.index, inplace = True)
val_test_df.drop(df0_train.index, inplace = True)
val_test_df.drop(df5_train.index, inplace = True)
val_test_df.drop(df3_train.index, inplace = True)

In [11]:
#taking 50% of the data that is left for the val df
df4_val = val_test_df[val_test_df["label"] == 4].sample(frac = .5)
df6_val = val_test_df[val_test_df["label"] == 6].sample(frac = .5)
df2_val = val_test_df[val_test_df["label"] == 2].sample(frac = .5)
df1_val = val_test_df[val_test_df["label"] == 1].sample(frac = .5)
df0_val = val_test_df[val_test_df["label"] == 0].sample(frac = .5)
df5_val = val_test_df[val_test_df["label"] == 5].sample(frac = .5)
df3_val = val_test_df[val_test_df["label"] == 3].sample(frac = .5)

In [12]:
#droping the val indices from the original data
#left over is the test df
test_df = val_test_df.drop(df4_val.index)
test_df.drop(df6_val.index, inplace = True)
test_df.drop(df2_val.index, inplace = True)
test_df.drop(df1_val.index, inplace = True)
test_df.drop(df0_val.index, inplace = True)
test_df.drop(df5_val.index, inplace = True)
test_df.drop(df3_val.index, inplace = True)

In [13]:
test_df.reset_index(inplace = True)

In [14]:
#combine all val classes
val_df = df4_val.append(df6_val).append(df2_val).append(df1_val).append(df0_val).append(df5_val).append(df3_val)
val_df.reset_index(inplace = True)

In [15]:
#combine all train classes
train_df = df4_train.append(df6_train).append(df2_train).append(df1_train).append(df0_train).append(df5_train).append(df3_train)
train_df.reset_index(inplace = True)

In [16]:
test_df["label"].value_counts()

4    1677
6     279
2     275
1     129
0      81
5      35
3      29
Name: label, dtype: int64

In [17]:
val_df["label"].value_counts()

4    1676
6     278
2     274
1     128
0      82
5      36
3      28
Name: label, dtype: int64

In [18]:
train_df["label"].value_counts()

4    3352
6     556
2     550
1     257
0     164
5      71
3      58
Name: label, dtype: int64

In [19]:
from keras.preprocessing.image import ImageDataGenerator

im_gen = ImageDataGenerator(
        rotation_range = 360,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        vertical_flip = True
)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


> Code to separate into labels and then save to folders:

In [31]:
#this iterates through each image in the training dataset created above by label and adds the image to the directory
for i in range(7):
    df = train_df[train_df["label"] == i]
    for j in df["image_id"].values:
        if int(j.replace("ISIC_00","")) < 29_306:
            image1 = Image.open(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/HAM10000_images_part_1/{j}.jpg")
            image1.save(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/image_data_train/type_{i}/{j}.jpg")

        else:
            image1 = Image.open(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/HAM10000_images_part_2/{j}.jpg")
            image1.save(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/image_data_train/type_{i}/{j}.jpg")

            
#below code was added to create an augmented training data folder, but was not used
    count = 0
    while count < (train_df["label"].shape[0]/df.shape[0]): #balancing the classes
        for s in df["image_id"].values:
        
            str_count = str(count)
            if int(s.replace("ISIC_00","")) < 29_306:
                #find image in one folder
                image2 = cv2.imread(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/HAM10000_images_part_1/{s}.jpg")
                #create a random transformation
                im_dict = im_gen.get_random_transform((450,600,3))
                #apply to image
                new_im = im_gen.apply_transform(image2, im_dict)
                image2 = Image.fromarray(new_im)
                #save to augmented folder
                image2.save(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/image_data_train_augmented/type_{i}/{s}{str_count}.jpg")
            else:
                #same thing as above but pulling from the second image folder
                image2 = cv2.imread(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/HAM10000_images_part_2/{s}.jpg")
                im_dict = im_gen.get_random_transform((450,600,3))
                new_im = im_gen.apply_transform(image2, im_dict)
                image2 = Image.fromarray(new_im)
                image2.save(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/image_data_train_augmented/type_{i}/{s}{str_count}.jpg")
        count += 1

In [21]:
#creating the test directory
for i in range(7):
    df = test_df[test_df["label"] == i]
    for j in df["image_id"].values:
        if int(j.replace("ISIC_00","")) < 29_306:
            image = Image.open(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/HAM10000_images_part_1/{j}.jpg")
            image.save(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/image_data_test//type_{i}/{j}.jpg")
        else:
            image = Image.open(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/HAM10000_images_part_2/{j}.jpg")
            image.save(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/image_data_test//type_{i}/{j}.jpg")

In [22]:
#creating the validation directory
for i in range(7):
    df = val_df[val_df["label"] == i]
    for j in df["image_id"].values:
        if int(j.replace("ISIC_00","")) < 29_306:
            image = Image.open(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/HAM10000_images_part_1/{j}.jpg")
            image.save(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/image_data_validation//type_{i}/{j}.jpg")
        else:
            image = Image.open(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/HAM10000_images_part_2/{j}.jpg")
            image.save(f"C:/Users/Vansh Aggarwal/Desktop/ai skin pro run/image_data_validation//type_{i}/{j}.jpg")